In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
        .master('local[*]')\
        .appName('hw')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 01:09:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/06 01:09:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-06.parquet

--2023-03-01 07:09:38--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-06.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.73, 52.84.160.84, 52.84.160.116, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394114750 (376M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-06.parquet’

fhvhv_tripdata_2021 100%[===================>] 375.86M  56.9MB/s    in 7.0s    

2023-03-01 07:09:46 (53.7 MB/s) - ‘fhvhv_tripdata_2021-06.parquet’ saved [394114750/394114750]



In [4]:
!ls


data				homework_week5.ipynb
download_data.sh		pyspark_latest.ipynb
fhvhv				spark-warehouse
fhvhv_tripdata_2021-01.csv	spark_sql.ipynb
fhvhv_tripdata_2021-01.parquet	taxi+_zone_lookup.csv
fhvhv_tripdata_2021-06.parquet	test_pyspark.ipynb
head.csv			zones


In [13]:
!wc -l 'fhvhv_tripdata_2021-06.parquet'

1278730 fhvhv_tripdata_2021-06.parquet


In [16]:
df = spark.read.format('parquet')\
    .load('fhvhv_tripdata_2021-06.parquet')

In [17]:
df.repartition(12).write.format('parquet').mode('overwrite').save('fhvhv/2021/06')

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [7]:
df = spark.read.parquet('fhvhv/2021/06')

In [8]:
df\
.select ('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID')\
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-06-27 08:58:38|2021-06-27 09:19:15|         255|         186|
|2021-06-06 15:19:30|2021-06-06 15:51:26|          95|         263|
|2021-06-13 16:23:14|2021-06-13 16:31:11|          76|          76|
|2021-06-24 07:01:30|2021-06-24 07:58:43|          76|          33|
|2021-06-10 00:03:03|2021-06-10 00:21:36|         148|          49|
|2021-06-10 04:49:00|2021-06-10 05:10:34|         116|          90|
|2021-06-03 02:24:12|2021-06-03 02:37:11|         248|         126|
|2021-06-08 10:48:56|2021-06-08 10:56:39|         185|         248|
|2021-06-30 00:01:04|2021-06-30 00:13:09|         164|          87|
|2021-06-26 13:19:38|2021-06-26 13:34:36|         164|          79|
|2021-06-06 14:04:55|2021-06-06 14:18:21|         117|         201|
|2021-06-01 02:01:41|2021-06-01 02:19:07|       

In [9]:
from pyspark.sql import functions as F

df_trip_data =df\
.withColumn('pickup_date',F.to_date(df.pickup_datetime))\
.withColumn('dropoff_date',F.to_date(df.dropoff_datetime))\
.select ('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','pickup_date','dropoff_date','trip_miles','trip_time')


In [10]:
df\
.select ('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_time')\
.show()

+-------------------+-------------------+------------+------------+---------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_time|
+-------------------+-------------------+------------+------------+---------+
|2021-06-27 08:58:38|2021-06-27 09:19:15|         255|         186|     1237|
|2021-06-06 15:19:30|2021-06-06 15:51:26|          95|         263|     1916|
|2021-06-13 16:23:14|2021-06-13 16:31:11|          76|          76|      477|
|2021-06-24 07:01:30|2021-06-24 07:58:43|          76|          33|     3433|
|2021-06-10 00:03:03|2021-06-10 00:21:36|         148|          49|     1113|
|2021-06-10 04:49:00|2021-06-10 05:10:34|         116|          90|     1294|
|2021-06-03 02:24:12|2021-06-03 02:37:11|         248|         126|      779|
|2021-06-08 10:48:56|2021-06-08 10:56:39|         185|         248|      464|
|2021-06-30 00:01:04|2021-06-30 00:13:09|         164|          87|      725|
|2021-06-26 13:19:38|2021-06-26 13:34:36|         164|          

In [55]:
df_trip_data.printSchema()

root
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)



In [12]:
df_trip_data.createOrReplaceTempView('trip_data')

23/03/06 01:10:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
spark.sql ("""
select  
        count(1) 
from 
        trip_data 
        where pickup_date ='2021-06-15'
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



In [14]:
spark.sql ("""
select  
        max(trip_time)
from 
        trip_data 

      
""").show()

+--------------+
|max(trip_time)|
+--------------+
|        240764|
+--------------+



In [15]:
df_zones = spark.read.parquet('../notebooks/zones')

In [18]:
df_zones.createOrReplaceTempView('zones')

In [20]:
spark.sql (
"""
select  
        zone , count(1)
from 
        trip_data a
inner join
        zones b
on

a.PULocationID = b.LocationID

group by 1
order by 2 desc

"""
).show()

+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
| Crown Heights North|  231279|
|        East Village|  221244|
|         JFK Airport|  188867|
|      Bushwick South|  187929|
|       East New York|  186780|
|TriBeCa/Civic Center|  164344|
|   LaGuardia Airport|  161596|
|            Union Sq|  158937|
|        West Village|  154698|
|             Astoria|  152493|
|     Lower East Side|  151020|
|        East Chelsea|  147673|
|Central Harlem North|  146402|
|Williamsburg (Nor...|  143683|
|          Park Slope|  143594|
|  Stuyvesant Heights|  141427|
|        Clinton East|  139611|
|West Chelsea/Huds...|  139431|
|             Bedford|  138428|
|         Murray Hill|  137879|
+--------------------+--------+
only showing top 20 rows

